\newpage

1. Using the UCI heart disease dataset, the project is trying to develop a predictive classification model to determine whether a patient has a heart problem based on 13 medical and demographical features. The response variable num, originally ranges from 0 to 4, is conveted to a binary variable that indicates the heart disease's presense. 0 represents no heart disease, and 1 (combining 1-4) represents its presense. Thus this project becomes a binary classification probelm with the objective of accurately predict the presense of heart disease.

In [2]:
#Load necessary libraries

from ucimlrepo import fetch_ucirepo 

import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
  


In [3]:
# fetch dataset 
heart_disease = fetch_ucirepo(id=45) 

df = pd.concat([heart_disease.data.features, heart_disease.data.targets], axis=1)

2. We applied one-hot encoding to all categorical variables to convert them into numerical form. Numerical features are standardized using z-score normalization. These transformations ensure that all input features are on the comparable scale, this is critically important to algorithms that are sensitive to feature magnitude. 

In [ ]:
#2

categorical_cols = ['sex', 'cp', 'fbs', 'restecg', 'exang', 'slope', 'ca', 'thal']
numerical_cols = [col for col in df.columns if col not in categorical_cols + ['num']]
df = pd.get_dummies(df, columns=categorical_cols, drop_first=True)
scaler = StandardScaler()
df[numerical_cols] = scaler.fit_transform(df[numerical_cols])

In [5]:
#4
df['target'] = df['num'].apply(lambda x: 1 if x > 0 else 0)
df.drop(columns='num', inplace=True)